# 基本回測範例

In [5]:
'''
回測是金融術語，利用歷史數據測試某個交易策略或模型的有效性。
通常在量化交易中使用，以評估策略在過去情境下的表現，從而推測其未來的潛在表現。
'''
import os
import pandas as pd
import yfinance as yf

'''自訂函數'''


# 取得資料：本地或是 Yahoo
def getDataFM(prod, st, en):
    # 檔案路徑
    bakfile = f"data/YF_{prod}_{st}_{en}_stock_daily_adj.csv"
    # 假如已經有 Excel 資料檔案
    if os.path.exists(bakfile):
        # 讀取 Excel 資料檔案
        data = pd.read_csv(bakfile, index_col="Date", parse_dates=True)
    # 假如 Excel 資料檔案不存在
    else:
        # 透過 yf 下載
        data = yf.download(f"{prod}.TW", start=st, end=en)
        # 下載的資料是英文的，處理欄位名稱，轉換為小寫
        # 轉換部分不包含索引，所以日期還是大寫開頭的 Date
        data.columns = [i.lower() for i in data.columns]
        # 假如下載後沒有資料
        if data.empty:
            print("無法從 Yahoo Finance 下載資料")
            # 傳回一個空的 pd
            return pd.DataFrame()
        # 假如資料 data 不是空的，依照指定路徑存擋
        data.to_csv(bakfile)
    # 存檔後，傳回資料
    return data


# 取得回測資料
data = getDataFM("0050", "2013-01-01", "2023-12-15")

# 初始持有部位為零
position = 0
# 開始回測
for i in range(data.shape[0] - 1):
    '''取得策略會應用到的變數'''
    # 取當前資料做為進場資料
    c_time = data.index[i]
    # 取當前資料的 low, high, close, open 並轉換為數字型態
    c_low = data.loc[c_time, "low"]
    c_high = data.loc[c_time, "high"]
    c_close = data.loc[c_time, "close"]
    c_open = data.loc[c_time, "open"]
    # 取下一期資料做為進場資料
    n_time = data.index[i + 1]
    n_open = data.loc[n_time, "open"]

    # 進場程序
    if position == 0:
        # 進場邏輯
        if c_close > c_open and (c_close - c_open) * 2 < (c_open - c_low):
            position = 1
            order_i = i
            order_time = n_time
            order_price = n_open
            order_unit = 1
            print(
                c_time,
                "觸發進場訊號 隔日進場",
                order_time,
                "進場價",
                order_price,
                "進場",
                order_unit,
                "單位",
            )
    # 出場程序
    elif position == 1:
        # 出場邏輯
        if i > order_i + 3 and c_close > c_open:
            position = 0
            cover_time = n_time
            cover_price = n_open
            print(c_time, "觸發出場訊號 隔日出場", cover_time, "出場價", cover_price)


2009-01-05 00:00:00 觸發進場訊號 隔日進場 2009-01-06 00:00:00 進場價 36.59000015258789 進場 1 單位
2009-01-12 00:00:00 觸發出場訊號 隔日出場 2009-01-13 00:00:00 出場價 34.189998626708984
2009-01-21 00:00:00 觸發進場訊號 隔日進場 2009-02-02 00:00:00 進場價 30.399999618530277 進場 1 單位
2009-02-06 00:00:00 觸發出場訊號 隔日出場 2009-02-09 00:00:00 出場價 29.950000762939453
2009-02-26 00:00:00 觸發進場訊號 隔日進場 2009-02-27 00:00:00 進場價 30.51000022888184 進場 1 單位
2009-03-04 00:00:00 觸發出場訊號 隔日出場 2009-03-05 00:00:00 出場價 35.040000915527344
2009-04-07 00:00:00 觸發進場訊號 隔日進場 2009-04-08 00:00:00 進場價 40.0 進場 1 單位
2009-04-13 00:00:00 觸發出場訊號 隔日出場 2009-04-14 00:00:00 出場價 42.9900016784668
2009-06-16 00:00:00 觸發進場訊號 隔日進場 2009-06-17 00:00:00 進場價 43.04999923706055 進場 1 單位
2009-06-22 00:00:00 觸發出場訊號 隔日出場 2009-06-23 00:00:00 出場價 42.84999847412109
2009-07-20 00:00:00 觸發進場訊號 隔日進場 2009-07-21 00:00:00 進場價 48.5 進場 1 單位
2009-07-27 00:00:00 觸發出場訊號 隔日出場 2009-07-28 00:00:00 出場價 48.83000183105469
2009-08-03 00:00:00 觸發進場訊號 隔日進場 2009-08-04 00:00:00 進場價 49.58000183105469 進場 1 單位
2009-